<h1 style="text-align:center;">Enable Secure Boot </h1>

<div class="alert alert-block alert-info" style="border-radius: 20px;">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>enabling secure boot</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3>2.This is our enable_secure_boot function overview, in the end this will give us with secure boot related information.</h3><br>
    <i>Let me give you a walk through of it!</i>
    <br>
There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to obtain the secure boot information.

1. In the <b>enable_secure_boot</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. In the ILO rest api the information about secureboot is stored inside the <b>systems json object</b>.
    Through systems_response variable  we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the secure boot about iLO system.
    <br>

3. In the systems response object we are iterating on all the members with different <b>odata_id</b> .
    Then again an http GET request is made and response is stored in <b>systems_members_response</b>.
    The secure boot information is ultimately present in the <b>systems_members_response</b>.
    <br>

4. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if secure boot is present in <b>@odata.type</b> attribute then we will store information about secure boot in secure_boot_uri and break out of iteration.
    <br>

Ultimately if secure_boot_uri was obtained without any errors then get the information at the relevant uri and print out that information .

 </div>

In [ ]:
def enable_secure_boot(_redfishobj, secure_boot_enable):

    secure_boot_uri = None
    secure_boot_data = None
    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        secure_boot_uri = systems_members_response.obj['SecureBoot']['@odata.id']
        secure_boot_data = _redfishobj.get(secure_boot_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#SecureBoot.' in instance['@odata.type']:
                secure_boot_uri = instance['@odata.id']
                secure_boot_data = _redfishobj.get(secure_boot_uri)

    if secure_boot_data:
        print("\n\nShowing Secure Boot properties before changes:\n\n")
        print(json.dumps(secure_boot_data.dict, indent=4, sort_keys=True))

    if secure_boot_uri:
        body = {'SecureBootEnable': secure_boot_enable}
        resp = _redfishobj.patch(secure_boot_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception:
                sys.stderr.write("A response error occurred, unable to access iLO Extended " \
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("\nSuccess!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            print("\n\nShowing Secure Boot properties after changes:\n\n")
            secure_boot_data = _redfishobj.get(secure_boot_uri)
            print(json.dumps(secure_boot_data.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <p>Please enter your login credentials here to proceed further:</p>
    <ul>
        <li>Once your credentials are entered, a Redfish object is created with the help of the RedfishClient class, and a session is created to log you in to the server.</li>
        <li>If the server is down or unreachable, we get a "server not reachable" message.</li>
        <li>If not, we make a call to the <b>enable_secure_boot</b> function to obtain secure boot-related information of the system.</li>
        <li>After getting the required information, <i><b>Redfish.logout()</b></i> closes the particular session and logs us out of the system.</li>
    </ul>
</div>


<div class = "alert alert-block alert-warning" style="border-radius: 20px;">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
   SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    #Properties:
    #secure boot enable property
    SECURE_BOOT_ENABLE = True
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    enable_secure_boot(REDFISHOBJ, SECURE_BOOT_ENABLE)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>